# Matrices convolution using Winograd's algorithm

In this notebook, we explore Winograd's convolution. We use the matrices as defined by Lavin and Gray (2015) doi :     
https://doi.org/10.48550/arXiv.1509.09308

In [1]:
import numpy as np

## First example

$4\times 4$ matrix with a kernel of $3\times 3$

### Input and Kernel

In [23]:
D = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]], dtype=np.float32)

In [24]:
D

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.],
       [ 9., 10., 11., 12.],
       [13., 14., 15., 16.]], dtype=float32)

In [25]:
g = np.array([[1, 0, -1], [2, 0, 2], [1, 0, -1]], dtype=np.float32)

In [26]:
g

array([[ 1.,  0., -1.],
       [ 2.,  0.,  2.],
       [ 1.,  0., -1.]], dtype=float32)

### Winograd matrices

In [27]:
G = np.array([[1, 0, 0], [1/2, 1/2, 1/2], [1/2, -1/2, 1/2], [0, 0, 1]], dtype=np.float32)

In [28]:
G

array([[ 1. ,  0. ,  0. ],
       [ 0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5],
       [ 0. ,  0. ,  1. ]], dtype=float32)

In [29]:
Bt = np.array([[1, 0, -1, 0], [0, 1, 1, 0], [0, -1, 1, 0], [0, 1, 0, -1]], dtype=np.float32)

In [30]:
Bt

array([[ 1.,  0., -1.,  0.],
       [ 0.,  1.,  1.,  0.],
       [ 0., -1.,  1.,  0.],
       [ 0.,  1.,  0., -1.]], dtype=float32)

In [31]:
At = np.array([[1, 1, 1, 0], [0, 1, -1, -1]], dtype=np.float32)

In [32]:
At

array([[ 1.,  1.,  1.,  0.],
       [ 0.,  1., -1., -1.]], dtype=float32)

### Intermediate matrices

In [33]:
G.shape, g.shape, G.T.shape

((4, 3), (3, 3), (3, 4))

In [34]:
U = G @ g @ G.T

In [35]:
U

array([[ 1.,  0.,  0., -1.],
       [ 2.,  1.,  1.,  0.],
       [ 0., -1., -1., -2.],
       [ 1.,  0.,  0., -1.]], dtype=float32)

In [36]:
V = Bt @ D @ Bt.T

In [37]:
V

array([[  0., -16.,   0.,   0.],
       [ -4.,  34.,   2.,  -4.],
       [  0.,   8.,   0.,   0.],
       [  0., -16.,   0.,   0.]], dtype=float32)

In [38]:
M = U * V

In [39]:
M

array([[ 0., -0.,  0., -0.],
       [-8., 34.,  2., -0.],
       [ 0., -8., -0., -0.],
       [ 0., -0.,  0., -0.]], dtype=float32)

### Output

In [40]:
Y = At @ M @ At.T

In [64]:
1.4 * Y

array([[28.      , 33.6     ],
       [50.399998, 56.      ]], dtype=float32)

### Verification

In [65]:
import scipy

In [71]:
scipy.signal.convolve2d(D, g, mode='valid')

array([[28., 32.],
       [44., 48.]], dtype=float32)

## Generate A, B, and G

We can use https://github.com/andravin/wincnn that automatically generates the above matrices.

In [1]:
from __future__ import print_function
from sympy import symbols, Matrix, Poly, zeros, eye, Indexed, simplify, IndexedBase, init_printing, pprint
from operator import mul
from functools import reduce

def At(a,m,n):
    return Matrix(m, n, lambda i,j: a[i]**j)

def A(a,m,n):
    return At(a, m-1, n).row_insert(m-1, Matrix(1, n, lambda i,j: 1 if j==n-1 else 0))

def T(a,n):
    return Matrix(Matrix.eye(n).col_insert(n, Matrix(n, 1, lambda i,j: -a[i]**n)))

def Lx(a,n):
    x=symbols('x')
    return Matrix(n, 1, lambda i,j: Poly((reduce(mul, ((x-a[k] if k!=i else 1) for k in range(0,n)), 1)).expand(basic=True), x))

def F(a,n):
    return Matrix(n, 1, lambda i,j: reduce(mul, ((a[i]-a[k] if k!=i else 1) for k in range(0,n)), 1))

def Fdiag(a,n):
    f=F(a,n)
    return Matrix(n, n, lambda i,j: (f[i,0] if i==j else 0))

def FdiagPlus1(a,n):
    f = Fdiag(a,n-1)
    f = f.col_insert(n-1, zeros(n-1,1))
    f = f.row_insert(n-1, Matrix(1,n, lambda i,j: (1 if j==n-1 else 0)))
    return f

def L(a,n):
    lx = Lx(a,n)
    f = F(a, n)
    return Matrix(n, n, lambda i,j: lx[i, 0].nth(j)/f[i]).T

def Bt(a,n):
    return L(a,n)*T(a,n)

def B(a,n):
    return Bt(a,n-1).row_insert(n-1, Matrix(1, n, lambda i,j: 1 if j==n-1 else 0))

FractionsInG=0
FractionsInA=1
FractionsInB=2
FractionsInF=3

def cookToomFilter(a,n,r,fractionsIn=FractionsInG):
    alpha = n+r-1
    f = FdiagPlus1(a,alpha)
    if f[0,0] < 0:
        f[0,:] *= -1
    if fractionsIn == FractionsInG:
        AT = A(a,alpha,n).T
        G = (A(a,alpha,r).T*f**(-1)).T
        BT = f * B(a,alpha).T
    elif fractionsIn == FractionsInA:
        BT = f * B(a,alpha).T
        G = A(a,alpha,r)
        AT = (A(a,alpha,n)).T*f**(-1)
    elif fractionsIn == FractionsInB:
        AT = A(a,alpha,n).T
        G = A(a,alpha,r)
        BT = B(a,alpha).T
    else:
        AT = A(a,alpha,n).T
        G = A(a,alpha,r)
        BT = f * B(a,alpha).T
    return (AT,G,BT,f)


def filterVerify(n, r, AT, G, BT):

    alpha = n+r-1

    di = IndexedBase('d')
    gi = IndexedBase('g')
    d = Matrix(alpha, 1, lambda i,j: di[i])
    g = Matrix(r, 1, lambda i,j: gi[i])

    V = BT*d
    U = G*g
    M = U.multiply_elementwise(V)
    Y = simplify(AT*M)

    return Y

def convolutionVerify(n, r, B, G, A):

    di = IndexedBase('d')
    gi = IndexedBase('g')

    d = Matrix(n, 1, lambda i,j: di[i])
    g = Matrix(r, 1, lambda i,j: gi[i])

    V = A*d
    U = G*g
    M = U.multiply_elementwise(V)
    Y = simplify(B*M)

    return Y

def showCookToomFilter(a,n,r,fractionsIn=FractionsInG):

    AT,G,BT,f = cookToomFilter(a,n,r,fractionsIn)

    print ("AT = ")
    pprint(AT)
    print ("")

    print ("G = ")
    pprint(G)
    print ("")

    print ("BT = ")
    pprint(BT)
    print ("")

    if fractionsIn != FractionsInF:
        print ("FIR filter: AT*((G*g)(BT*d)) =")
        pprint(filterVerify(n,r,AT,G,BT))
        print ("")

    if fractionsIn == FractionsInF:
        print ("fractions = ")
        pprint(f)
        print ("")

def showCookToomConvolution(a,n,r,fractionsIn=FractionsInG):

    AT,G,BT,f = cookToomFilter(a,n,r,fractionsIn)

    B = BT.transpose()
    A = AT.transpose()

    print ("A = ")
    pprint(A)
    print ("")

    print ("G = ")
    pprint(G)
    print ("")

    print ("B = ")
    pprint(B)
    print ("")

    if fractionsIn != FractionsInF:
        print ("Linear Convolution: B*((G*g)(A*d)) =")
        pprint(convolutionVerify(n,r,B,G,A))
        print ("")

    if fractionsIn == FractionsInF:
        print ("fractions = ")
        pprint(f)
        print ("")


In [6]:
showCookToomConvolution((0, 1, -1, 2, -2), 3, 3)

A = 
⎡1  0   0⎤
⎢        ⎥
⎢1  1   1⎥
⎢        ⎥
⎢1  -1  1⎥
⎢        ⎥
⎢1  2   4⎥
⎢        ⎥
⎣0  0   1⎦

G = 
⎡1/2    0     0  ⎤
⎢                ⎥
⎢-1/2  -1/2  -1/2⎥
⎢                ⎥
⎢-1/6  1/6   -1/6⎥
⎢                ⎥
⎢1/6   1/3   2/3 ⎥
⎢                ⎥
⎣ 0     0     1  ⎦

B = 
⎡2   0   0   0   0 ⎤
⎢                  ⎥
⎢-1  -2  2   -1  2 ⎥
⎢                  ⎥
⎢-2  -1  -3  0   -1⎥
⎢                  ⎥
⎢1   1   1   1   -2⎥
⎢                  ⎥
⎣0   0   0   0   1 ⎦

Linear Convolution: B*((G*g)(A*d)) =
⎡            d[0]⋅g[0]            ⎤
⎢                                 ⎥
⎢      d[0]⋅g[1] + d[1]⋅g[0]      ⎥
⎢                                 ⎥
⎢d[0]⋅g[2] + d[1]⋅g[1] + d[2]⋅g[0]⎥
⎢                                 ⎥
⎢      d[1]⋅g[2] + d[2]⋅g[1]      ⎥
⎢                                 ⎥
⎣            d[2]⋅g[2]            ⎦



/tmp/ipykernel_348864/2220159112.py:17: SymPyDeprecationWarning: 

non-Expr objects in a Matrix is deprecated. Matrix represents
a mathematical matrix. To represent a container of non-numeric
entities, Use a list of lists, TableForm, NumPy array, or some
other data structure instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-non-expr-in-matrix
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  return Matrix(n, 1, lambda i,j: Poly((reduce(mul, ((x-a[k] if k!=i else 1) for k in range(0,n)), 1)).expand(basic=True), x))
